<center>
<h1 style="color: pink;">Welcome to Publish Mate 😊</h1>
</center>

## `00` Download Dependencies

In [94]:
# !pip3 install -U "crewai[tools,agentops]"

In [95]:
# !pip3 install python-dotenv
# !pip3 install gcloud
# !pip3 install google-genai

## `01` Import Libraries

In [96]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

In [97]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from crewai.llms.base_llm import BaseLLM

from pydantic import BaseModel, Field
from typing import List, Dict

import agentops
import json
import gcloud
import google.generativeai as genai
from dotenv import load_dotenv
from vertexai.preview.generative_models import Content, Part


## `02` load api key

In [98]:
load_dotenv()  # Load from .env

True

In [99]:
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") # replace by yours

In [100]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PROJECT_ID = os.getenv("PROJECT_ID")
PROJECT_NAME = os.getenv("PROJECT_NAME")

genai.configure(api_key=OPENAI_API_KEY)

## `03` Start AgentOps session

In [101]:
agentops.init(api_key=AGENTOPS_API_KEY,
               skip_auto_end_session=True, # Set to True to skip auto ending the session
               ) 

The link will help us to monitor our agents

### Make sure it works

In [102]:
# print("AgentOps session initialized.")
# print(agentops.session)  # optional, shows session info if available
# print(agentops.__dict__)

## `04` Intro of the Crew

In [103]:
intro_prompt = (
    "Welcome to PublishMate! I am your research assistant mate here to help you with your academic paper journey.\n"
    "I will guide you step-by-step to find trending topics, recent papers, summaries, "
    "research gaps, and help with paper writing. \nLet's get started!\n"
)

def welcome_message():
    print(intro_prompt)

# Run this at the very beginning
welcome_message()

Welcome to PublishMate! I am your research assistant mate here to help you with your academic paper journey.
I will guide you step-by-step to find trending topics, recent papers, summaries, research gaps, and help with paper writing. 
Let's get started!



## `05` Set Output dir

In [104]:
output_dir = './PublishMate_agent_ouput'
os.makedirs(output_dir, exist_ok=True)

## `06` LLM will be used

In [ ]:
basic_llm = LLM(
    # Corrected the model name from "gemini-2.0-flas" to "gemini-1.5-flash"
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    provider="google_ai_studio",
    api_key=os.environ["GEMINI_API_KEY"]
)

## `07` START AGENTS

### `7.1` Agent 1: Trending Topics Agent 

In [106]:
# !gcloud init

In [107]:
user_input = input("Enter your research field or keyword: ")

In [108]:
class TrendingTopicsOutput(BaseModel):
    topics: List[Dict[str, str]] = Field(..., title="Trending topics with description", min_items=1)

trending_topics_agent = Agent(
    role="Trending Topics Identification Agent",
    goal="\n".join([
        intro_prompt,
        "You are an expert research assistant that identifies the latest trending topics in a given academic field.",
        "Given a user-supplied research field or keyword, generate a detailed list of the top 3-5 trending topics reflecting recent advances and high interest areas."
    ]),
    backstory="Designed to guide users by providing the most relevant and current trending research topics in their specified field.",
    llm=basic_llm,
    verbose=True,
)

trending_topics_task = Task(
    description="\n".join([
        "User inputs a research field or keyword (e.g., 'AI agents', 'transformers').",
        "Provide a list of 3 to 5 trending topics with a brief description for each.",
        "Focus on recent research interests supported by publication trends.",
        "Output in JSON format with 'topics' as list of objects {name, description}."
    ]),
    expected_output="JSON object with list of trending topics and descriptions.",
    output_json=TrendingTopicsOutput,
    output_file=os.path.join(output_dir, "step_1_trending_topics.json"),
    agent=trending_topics_agent,
)

### `7.2` Agent 2: Recent Papers Retrieval Agent

In [109]:
class PaperInfo(BaseModel):
    title: str
    authors: str
    abstract: str
    year: str
    url: str

class RecentPapersOutput(BaseModel):
    topic_papers: Dict[str, List[PaperInfo]] = Field(..., title="Recent papers grouped by topic")

recent_papers_agent = Agent(
    role="Recent Papers Retrieval Agent",
    goal="\n".join([
        intro_prompt,
        "You are a research paper search assistant.",
        "Given a list of trending topics, retrieve 3 recent, relevant publications per topic.",
        "Select papers from reputable sources published within the last 2 years."
    ]),
    backstory="Fetches and organizes recent academic papers for user review.",
    llm=basic_llm,
    verbose=True,
)

recent_papers_task = Task(
    description="\n".join([
        "Input is a list of trending topics.",
        "For each topic, find 3 papers with title, authors, abstract, year, and link.",
        "Focus on papers from last 2 years from reputable conferences or journals.",
        "Output JSON grouped by topic."
    ]),
    expected_output="JSON with topics as keys and list of paper info objects as values.",
    output_json=RecentPapersOutput,
    output_file=os.path.join(output_dir, "step_2_recent_papers.json"),
    agent=recent_papers_agent,
)


### `7.3` Agent 3: Paper Summarization Agent

In [110]:
class PaperSummariesOutput(BaseModel):
    summaries: Dict[str, str] = Field(
        ..., 
        title="Paper title mapped to its summary", 
        description="Each item has 'title' and 'summary'."
    )

paper_summarization_agent = Agent(
    role="Academic Paper Summarization Agent",
    goal="\n".join([
        "Summarize each research paper into a detailed 120-150 word paragraph.",
        "Mention the full paper title before the summary.",
        "Focus on: main research problem, methodology, key findings, unique contributions.",
        "Highlight any datasets, models, or diagrams used (if mentioned).",
        "Avoid generic descriptions. Be specific about what the paper achieves."
    ]),  # <== FIXED: added comma here
    backstory="Provides clear and informative summaries to help users understand research papers quickly even if they are beginners.",
    llm=basic_llm,
    verbose=True,
)

paper_summarization_task = Task(
    description="\n".join([
        "Input is a list of papers with metadata and abstracts.",
        "Produce a summary for each paper highlighting key points and visuals if any.",
        "Output JSON mapping paper titles to summaries."
    ]),
    expected_output="JSON object mapping paper titles to summaries.",
    output_json=PaperSummariesOutput,
    output_file=os.path.join(output_dir, "step_3_paper_summaries.json"),
    agent=paper_summarization_agent,
)


## `08` Crew

In [111]:
# Define the Crew
crew_agents = Crew(
    name="PublishMate Crew",
    description="A crew of agents designed to assist with academic research and paper writing.",
    agents=[trending_topics_agent, recent_papers_agent, paper_summarization_agent],
    tasks=[trending_topics_task, recent_papers_task, paper_summarization_task],
)

result = crew_agents.kickoff()
print(result)

# Agent: Trending Topics Identification Agent
## Task: User inputs a research field or keyword (e.g., 'AI agents', 'transformers').
Provide a list of 3 to 5 trending topics with a brief description for each.
Focus on recent research interests supported by publication trends.
Output in JSON format with 'topics' as list of objects {name, description}.


# Agent: Trending Topics Identification Agent
## Final Answer: 
{
  "topics": [
    {
      "name": "Large Language Models (LLMs) for AI Agents",
      "description": "Research focuses on leveraging LLMs like GPT-3, LaMDA, and PaLM to enhance the capabilities of AI agents, particularly in areas such as natural language understanding, dialogue management, and complex reasoning.  This includes work on prompt engineering, few-shot learning, and fine-tuning LLMs for specific agent tasks.  A key area is improving the safety and reliability of LLM-powered agents."
    },
    {
      "name": "Multi-Agent Reinforcement Learning (MARL) for Collabo

🖇 AgentOps: Session Replay for default.session trace: https://app.agentops.ai/sessions?trace_id=8b38f42e62f4d254c9105a0718c77bb2


{'summaries': {'Chain-of-Thought Prompting Elicits Reasoning in Large Language Models': "This 2022 paper by Wei et al. addresses the limited reasoning capabilities of Large Language Models (LLMs).  The authors introduce 'chain-of-thought prompting,' a technique that improves LLM performance by providing a step-by-step reasoning process within the prompt itself.  Experiments across various LLMs and datasets demonstrate significant performance improvements on arithmetic, commonsense, and symbolic reasoning tasks.  The key contribution is the simplicity and effectiveness of chain-of-thought prompting in eliciting better reasoning from existing LLMs, without requiring model modifications.", 'Language Models are Few-Shot Learners': "Brown et al.'s 2020 paper introduces GPT-3, a 175-billion parameter language model.  The research focuses on demonstrating GPT-3's impressive few-shot learning capabilities.  The authors show that GPT-3 achieves strong performance on numerous tasks with minimal 